## Convolutional Neural Network (CNN) Example
This example is from the Udemy course titled "Deep Learning A-Z - Handson ANNs"

In [1]:
# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

In [2]:
# ensure that keras is installed
# $ conda install keras

In [3]:
# this is a binary classifier problem where we decide if an input picture is either cat or a dog.
# we have a few thousand pictures of dogs and cats in seperate folders. They are under "\cnn_dataset"
# we have 8,000 training images and 2,000 test images.

# Part 1 - Building the CNN

In [4]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [5]:
# Initialising the CNN
classifier = Sequential()

In [6]:
# Step 1 - Convolutional Layer
# start with 32 layers (max is 128), using 3x3 filters
# we will later force all our input images to 64x64x3 images (we are using CPUs so we will test with small images)
# since we are using TensorFlow instead of Theano, the order of parameters in input_shape is as follows.
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

In [7]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [8]:
# Adding a second convolutional layer
# the input is the output of the previous layer, so we do not need the 'input_shape' parameter
# we could have added a larger filter (64 instead of 32)
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [9]:
# Step 3 - Flattening
classifier.add(Flatten())

In [10]:
# Step 4 - Full connection
# this is a binary classifier problem where we decide if an input picture is either cat or a dog.
# the output is a single node.
classifier.add(Dense(units = 128, activation = 'relu'))  # hidden layer of 128 nodes is chosen based on experience
classifier.add(Dense(units = 1, activation = 'sigmoid'))  # output layer is a single node

In [11]:
# Compiling the CNN
# this is a binary classifier problem where we decide if an input picture is either cat or a dog.
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Part 2 - Fitting the CNN to the images (image augmentation and preprocessing by Keras)
# check https://keras.io 

In [13]:
from keras.preprocessing.image import ImageDataGenerator

In [14]:
# augmentation enriches the training dataset
# see https://keras.io/preprocessing/image/
# we will use "flow_from_directory" example on that page
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,  # shear transformation (a.k.a transvection)
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [15]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
# we want to transform our training images into 64x64x3 resolution
training_set = train_datagen.flow_from_directory('cnn_dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [17]:
# we want to transform our test images into 64x64x3 resolution
test_set = test_datagen.flow_from_directory('cnn_dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [ ]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,  # no of images in the training dataset
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000) # no of images in the test dataset
# should it be 'nb_val_samples' instead of 'validation_steps' ?


Epoch 1/25
8000/8000 [==============================] - 5429s 679ms/step - loss: 0.3799 - accuracy: 0.8208 - val_loss: 0.6999 - val_accuracy: 0.7821
Epoch 2/25
3202/8000 [===========>..................] - ETA: 30:44 - loss: 0.1685 - accuracy: 0.9329

In [ ]:
# We should have small difference between the accuracy of the training set and test set.
# To improve the accuracy of our model, we could either add a new convolutional layer or add a new fully connected layer. 

In [ ]:
# # Part 3 - Making new predictions

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('cnn_dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
# this function creates the 3rd dimension of the image (which is the color)
test_image = image.img_to_array(test_image)
# the predict() method expects the image in 4 dimensions. The 4th dimension is a batch.
# So we have one batch that consists of one image here. 
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [ ]:
print(prediction)